In [ ]:
import scanpy as sc
import numpy as np
import pandas as pd
import milopy
import milopy.core as milo
import palettable
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
adata = sc.read_h5ad("/add/path/here/full_cohort.h5ad")

In [ ]:
refined_annotations = pd.read_csv("/add/path/here/refined_annotations.csv",index_col=0)

refined_annotations.columns = ["refined_annotations"]

In [ ]:
highlevel_refined = {"Hepatocyte": "Epithelial", 
                     "Carcinoma": "Carcinoma", 
                     "Fibroblast": "Fibroblast", 
                     "Quiescent endothelial cells": "Endothelial", 
                     "Smooth muscle": "Muscle", 
                     "Skeletal muscle": "Muscle",
                     "TAM2": "Myeloid", "TAM3": "Myeloid",
                     "TCD4": "Lymphoid", 
                     "Inflammatory CAF": "Fibroblast", 
                     "Adipose CAF": "Fibroblast",
                     "HGF-CAF": "Fibroblast",
                     "TAM1": "Myeloid", 
                     "Myeloid-HighMT": "Unknown/technical", 
                     "Angiogenic EC": "Endothelial", 
                     "Quiescent EC": "Endothelial", 
                     "Venous EC": "Endothelial",
                     "TCD8": "Lymphoid", 
                     "B": "Lymphoid", 
                     "DC": "Myeloid", 
                     "Hepatic EC": "Endothelial", 
                     "Kupffer cells": "Myeloid", 
                     "NK": "Lymphoid", 
                     "Treg": "Lymphoid", 
                     "StrMus-HighMT": "Unknown/technical", 
                     "T-HighMT": "Unknown/technical", 
                     "Mast": "Myeloid", 
                     "Adipocytes": "Stromal/Muscle", 
                     "Endo-HighMT": "Unknown/technical"}

In [ ]:
adata.obs = pd.concat([adata.obs,refined_annotations],axis=1)
adata.obs["highlevel_refined"] = adata.obs.refined_annotations.replace(highlevel_refined)

# Subset macrophage

## Comparing primary vs metastatic

In [ ]:
subadata = adata[(adata.obs.highlevel_refined=="Myeloid") & (adata.obs.Tumor==True)].copy()
sc.tl.pca(subadata)
sc.external.pp.harmony_integrate(subadata, key="sample_id", basis="X_pca", max_iter_harmony=20)
sc.pp.neighbors(subadata, use_rep="X_pca_harmony", n_neighbors=20)
sc.tl.umap(subadata)

In [ ]:
colorlist = palettable.colorbrewer.sequential.Greens_9.mpl_colors
ctlist = subadata.obs.refined_annotations.unique()
colormapping = {ct: colorlist[i+1] for i,ct in enumerate(ctlist)}
colormapping["Mixed"] = "whitesmoke"

In [ ]:
subadata.X = subadata.layers["counts"] 

In [ ]:
## Assign cells to neighbourhoods
milo.make_nhoods(subadata)

## Count cells from each sample in each nhood
milo.count_nhoods(subadata, sample_col="sample_id")

## Test for differential abundance between conditions
milo.DA_nhoods(subadata, design="~ Metastatic")

In [ ]:
fig, ax = plt.subplots(1,1)
milopy.utils.build_nhood_graph(subadata)
milopy.plot.plot_nhood_graph(subadata, alpha=0.1, min_size=4, ax=ax)
fig.savefig("figures/myeloid/DA_milo_myeloid_umap_fdr.png", dpi=200, bbox_inches="tight")

In [ ]:
milopy.utils.annotate_nhoods(subadata, anno_col='refined_annotations')

In [ ]:
plt.hist(subadata.uns['nhood_adata'].obs["nhood_annotation_frac"]);
plt.xlabel("celltype fraction")

In [ ]:
subadata.uns['nhood_adata'].obs.loc[subadata.uns['nhood_adata'].obs["nhood_annotation_frac"] < 0.6, "nhood_annotation"] = "Mixed"

In [ ]:
fig, ax = plt.subplots(1,1)
sns.swarmplot(data=subadata.uns['nhood_adata'].obs, x="nhood_annotation", 
              y="logFC", palette=colormapping, order=["DC", "Mast", "Kupffer cells", "TAM1", "TAM2"], ax=ax)
sns.boxplot(data=subadata.uns['nhood_adata'].obs, x="nhood_annotation", y="logFC",
                 showcaps=False,width=0.3,boxprops={'facecolor':'None','linewidth':1},
                 showfliers=False,whiskerprops={'linewidth':1}, order=["DC", "Mast", "Kupffer cells", "TAM1", "TAM2"],ax=ax)
ax.set_title("Differential abundance, Myeloid", fontsize=15, style="italic") 
ax.spines[["top","right"]].set_visible(False)
ax.spines[["bottom","left"]].set_linewidth(4)
ax.set_xticklabels(ax.get_xticklabels(), fontsize=15, rotation=45)
ax.set_xlabel("")
ax.set_yticklabels(ax.get_yticklabels(), fontsize=15)
ax.set_ylabel("log(FC)", fontsize=15)
ax.hlines(y=0, xmin=ax.get_xlim()[0], xmax=ax.get_xlim()[1], linestyles="dashed", color="grey")
fig.savefig("figures/myeloid/DA_milo_myeloid_boxplot.png", dpi=200, bbox_inches="tight")
fig.savefig("figures/myeloid/DA_milo_myeloid_boxplot.svg", dpi=200, bbox_inches="tight")

## Comparing normal vs tumor

In [ ]:
subadata = adata[(adata.obs.highlevel_refined=="Myeloid")].copy()
sc.tl.pca(subadata)
sc.external.pp.harmony_integrate(subadata, key="sample_id", basis="X_pca", max_iter_harmony=20)
sc.pp.neighbors(subadata, use_rep="X_pca_harmony", n_neighbors=20)
sc.tl.umap(subadata)

In [ ]:
colorlist = palettable.colorbrewer.sequential.Greens_9.mpl_colors
ctlist = subadata.obs.refined_annotations.unique()
colormapping = {ct: colorlist[i+1] for i,ct in enumerate(ctlist)}
colormapping["Mixed"] = "whitesmoke"

In [ ]:
subadata.X = subadata.layers["counts"] 

In [ ]:
## Assign cells to neighbourhoods
milo.make_nhoods(subadata)

## Count cells from each sample in each nhood
milo.count_nhoods(subadata, sample_col="sample_id")

## Test for differential abundance between conditions
milo.DA_nhoods(subadata, design="~ Tumor")

In [ ]:
fig, ax = plt.subplots(1,1)
milopy.utils.build_nhood_graph(subadata)
milopy.plot.plot_nhood_graph(subadata, alpha=0.1, min_size=4, ax=ax)
fig.savefig("figures/myeloid/DA_milo_myeloid_tumorstatus_umap_fdr.png", dpi=200, bbox_inches="tight")

In [ ]:
milopy.utils.annotate_nhoods(subadata, anno_col='refined_annotations')

In [ ]:
plt.hist(subadata.uns['nhood_adata'].obs["nhood_annotation_frac"]);
plt.xlabel("celltype fraction")

In [ ]:
subadata.uns['nhood_adata'].obs.loc[subadata.uns['nhood_adata'].obs["nhood_annotation_frac"] < 0.6, "nhood_annotation"] = "Mixed"

In [ ]:
fig, ax = plt.subplots(1,1)
sns.swarmplot(data=subadata.uns['nhood_adata'].obs, x="nhood_annotation", 
              y="logFC", palette=colormapping, order=["DC", "Mast", "Kupffer cells", "TAM1", "TAM2"], ax=ax)
sns.boxplot(data=subadata.uns['nhood_adata'].obs, x="nhood_annotation", y="logFC",
                 showcaps=False,width=0.3,boxprops={'facecolor':'None','linewidth':1},
                 showfliers=False,whiskerprops={'linewidth':1}, order=["DC", "Mast", "Kupffer cells", "TAM1", "TAM2"],ax=ax)
ax.set_title("Differential abundance, Myeloid", fontsize=15, style="italic") 
ax.spines[["top","right"]].set_visible(False)
ax.spines[["bottom","left"]].set_linewidth(4)
ax.set_xticklabels(ax.get_xticklabels(), fontsize=15, rotation=45)
ax.set_xlabel("")
ax.set_yticklabels(ax.get_yticklabels(), fontsize=15)
ax.set_ylabel("log(FC)", fontsize=15)
ax.hlines(y=0, xmin=ax.get_xlim()[0], xmax=ax.get_xlim()[1], linestyles="dashed", color="grey")
fig.savefig("figures/myeloid/DA_milo_myeloid_tumorstatus_boxplot.png", dpi=200, bbox_inches="tight")
fig.savefig("figures/myeloid/DA_milo_myeloid_tumorstatus_boxplot.svg", dpi=200, bbox_inches="tight")

# Subset lymphoid

In [ ]:
subadata = adata[(adata.obs.highlevel_refined=="Lymphoid") & (adata.obs.Tumor==True)].copy()
sc.tl.pca(subadata)
sc.external.pp.harmony_integrate(subadata, key="sample_id", basis="X_pca", max_iter_harmony=20)
sc.pp.neighbors(subadata, use_rep="X_pca_harmony", n_neighbors=20)
sc.tl.umap(subadata)

In [ ]:
colorlist = palettable.colorbrewer.sequential.Greys_9.mpl_colors
ctlist = subadata.obs.refined_annotations.unique()
colormapping = {ct: colorlist[i+2] for i,ct in enumerate(ctlist)}
colormapping["B"] = colorlist[8]
colormapping["Mixed"] = "whitesmoke"

In [ ]:
subadata.X = subadata.layers["counts"] 

subadata.obs["Condition"] = subadata.obs.sample_id.replace(clinical["Metastatic?"].to_dict())

In [ ]:
## Assign cells to neighbourhoods
milo.make_nhoods(subadata)

## Count cells from each sample in each nhood
milo.count_nhoods(subadata, sample_col="sample_id")

## Test for differential abundance between conditions
milo.DA_nhoods(subadata, design="~ Condition")

In [ ]:
fig, ax = plt.subplots(1,1)
milopy.utils.build_nhood_graph(subadata)
milopy.plot.plot_nhood_graph(subadata, alpha=0.1, min_size=4, ax=ax)
fig.savefig("figures/lymphoid/DA_milo_lymphoid_umap_fdr.png", dpi=200, bbox_inches="tight")

In [ ]:
fig= sc.pl.umap(subadata, color=["refined_annotations"], 
           ncols=1, frameon=False, palette=colormapping, return_fig=True)
fig.savefig("figures/lymphoid/umap_annotations_subsetmal.png",dpi=200,bbox_inches="tight")

In [ ]:
milopy.utils.annotate_nhoods(subadata, anno_col='refined_annotations')

In [ ]:
plt.hist(subadata.uns['nhood_adata'].obs["nhood_annotation_frac"]);
plt.xlabel("celltype fraction")

In [ ]:
subadata.uns['nhood_adata'].obs.loc[subadata.uns['nhood_adata'].obs["nhood_annotation_frac"] < 0.6, "nhood_annotation"] = "Mixed"

In [ ]:
fig, ax = plt.subplots(1,1)
sns.swarmplot(data=subadata.uns['nhood_adata'].obs, x="nhood_annotation", 
              y="logFC", palette=colormapping, order=["B", "NK", "TCD8", "TCD4", "Treg", "Mixed"], ax=ax)
sns.boxplot(data=subadata.uns['nhood_adata'].obs, x="nhood_annotation", y="logFC",
                 showcaps=False,width=0.3,boxprops={'facecolor':'None','linewidth':1},
                 showfliers=False,whiskerprops={'linewidth':1}, order=["B", "NK", "TCD8", "TCD4", "Treg", "Mixed"],ax=ax)
ax.set_title("Differential abundance, Lymphoid", fontsize=15, style="italic") 
ax.spines[["top","right"]].set_visible(False)
ax.spines[["bottom","left"]].set_linewidth(4)
ax.set_xticklabels(ax.get_xticklabels(), fontsize=15, rotation=45)
ax.set_xlabel("")
ax.set_yticklabels(ax.get_yticklabels(), fontsize=15)
ax.set_ylabel("log(FC)", fontsize=15)
ax.hlines(y=0, xmin=ax.get_xlim()[0], xmax=ax.get_xlim()[1], linestyles="dashed", color="grey")
fig.savefig("figures/lymphoid/DA_milo_lymphoid_boxplot.png", dpi=200, bbox_inches="tight")

# Subset fibroblast

In [ ]:
subadata = adata[(adata.obs.highlevel_refined=="Fibroblast") & (adata.obs.Tumor==True)].copy()
sc.tl.pca(subadata)
sc.external.pp.harmony_integrate(subadata, key="sample_id", basis="X_pca", max_iter_harmony=20)
sc.pp.neighbors(subadata, use_rep="X_pca_harmony", n_neighbors=20)
sc.tl.umap(subadata)

In [ ]:
colorlist = palettable.colorbrewer.sequential.Oranges_5.mpl_colors
ctlist = subadata.obs.refined_annotations.unique()
colormapping = {ct: colorlist[i+1] for i,ct in enumerate(ctlist)}
colormapping["Mixed"] = "whitesmoke"

In [ ]:
subadata.X = subadata.layers["counts"] 

subadata.obs["Condition"] = subadata.obs.sample_id.replace(clinical["Metastatic?"].to_dict())

In [ ]:
## Assign cells to neighbourhoods
milo.make_nhoods(subadata)

## Count cells from each sample in each nhood
milo.count_nhoods(subadata, sample_col="sample_id")

## Test for differential abundance between conditions
milo.DA_nhoods(subadata, design="~ Condition")

In [ ]:
fig, ax = plt.subplots(1,1)
milopy.utils.build_nhood_graph(subadata)
milopy.plot.plot_nhood_graph(subadata, alpha=0.1, min_size=4, ax=ax)
fig.savefig("figures/fibroblast/DA_milo_fibroblast_umap_fdr.png", dpi=200, bbox_inches="tight")

In [ ]:
fig= sc.pl.umap(subadata, color=["refined_annotations"], 
           ncols=1, frameon=False, palette=colormapping, return_fig=True)
fig.savefig("figures/fibroblast/umap_annotations_subsetmal.png",dpi=200,bbox_inches="tight")

In [ ]:
milopy.utils.annotate_nhoods(subadata, anno_col='refined_annotations')

In [ ]:
plt.hist(subadata.uns['nhood_adata'].obs["nhood_annotation_frac"]);
plt.xlabel("celltype fraction")

In [ ]:
subadata.uns['nhood_adata'].obs.loc[subadata.uns['nhood_adata'].obs["nhood_annotation_frac"] < 0.6, "nhood_annotation"] = "Mixed"

In [ ]:
fig, ax = plt.subplots(1,1)
sns.swarmplot(data=subadata.uns['nhood_adata'].obs, x="nhood_annotation", 
              y="logFC", palette=colormapping, order=["Fibroblast", "Inflammatory CAF", "Adipose CAF", "HGF-CAF", "Mixed"], ax=ax)
sns.boxplot(data=subadata.uns['nhood_adata'].obs, x="nhood_annotation", y="logFC",
                 showcaps=False,width=0.3,boxprops={'facecolor':'None','linewidth':1},
                 showfliers=False,whiskerprops={'linewidth':1}, order=["Fibroblast", "Inflammatory CAF", "Adipose CAF", "HGF-CAF", "Mixed"],
            ax=ax)
ax.set_title("Differential abundance, Fibroblast", fontsize=15, style="italic") 
ax.spines[["top","right"]].set_visible(False)
ax.spines[["bottom","left"]].set_linewidth(4)
ax.set_xticklabels(ax.get_xticklabels(), fontsize=15, rotation=45)
ax.set_xlabel("")
ax.set_yticklabels(ax.get_yticklabels(), fontsize=15)
ax.set_ylabel("log(FC)", fontsize=15)
ax.hlines(y=0, xmin=ax.get_xlim()[0], xmax=ax.get_xlim()[1], linestyles="dashed", color="grey")
fig.savefig("figures/fibroblast/DA_milo_fibroblast_boxplot.png", dpi=200, bbox_inches="tight")
fig.savefig("figures/fibroblast/DA_milo_fibroblast_boxplot.svg", dpi=200, bbox_inches="tight")